# function to extract data and create a dataframe

In [ ]:
import sys
import os

sys.path.append(os.path.abspath(".."))

import xarray as xr
import pandas as pd 
import polars as pl
from config import * 

file_path = os.path.join(NETCDF_FILE, "01012000_rrs_mediterranean_cmems_monthly.nc")
data = xr.open_dataset(file_path, decode_times=True)

csv_path = os.path.join(CSV_DIR, "jan2000.csv")
df_csv = data.to_dataframe().reset_index()
df_csv.to_csv(csv_path, index=False)

df_rrs = data.to_dataframe().dropna().reset_index()
df_lat_lon = data.to_dataframe().reset_index()

## there are NaNs values and the time is 2002-12-31

In [ ]:
df_rrs['time'] = pd.Timestamp("2000-01-15")
df_lat_lon['time'] = pd.Timestamp("2000-01-15")
df_pl = pl.from_pandas(df_rrs)
df = df_lat_lon
print(df.columns)

In [ ]:
import cartopy.crs as ccrs
import matplotlib.pyplot as plt
import cartopy.feature as cfeature
import os

save_var = True
figure_path = os.path.join(FIGURE_DIR, 'jan_2000_ref.png')

fig = plt.figure(figsize=(18, 12))

for idx, wl in enumerate(WAVELENGTH_DIM):
    ax = plt.subplot(2, 3, idx + 1, projection=ccrs.PlateCarree())
    scatter = ax.scatter(df['lon'], df['lat'], c=df[f'RRS{wl}'], s=1, cmap='viridis',
                 alpha=0.8, transform=ccrs.PlateCarree(), vmin=df[f'RRS{wl}'].quantile(0.01),
                 vmax=df[f'RRS{wl}'].quantile(0.99))
    cbar = plt.colorbar(scatter, ax=ax, orientation='horizontal', 
                       pad=0.01, shrink=0.7)
    cbar.set_label(f'{wl} nm Reflectance', fontsize=12)
    cbar.ax.tick_params(labelsize=8)
    ax.coastlines(resolution='10m', linewidth=0.2, color='black')
    ax.add_feature(cfeature.LAND, facecolor='lightgray', alpha=0.3)
    ax.set_title(f'RRS {wl} nm', fontsize=11, fontweight='bold')

fig.suptitle(f'Mediterranean Sea Remote Sensing Reflectance, January 2000', 
                fontsize=14, fontweight='bold', y=0.78)
# plt.subplots_adjust(hspace=0.1) # different method to decrease space between subplots
plt.tight_layout(rect=[0, 0, 1, 0.95], h_pad=0.5)
if save_var:
    plt.savefig(figure_path, bbox_inches='tight', format='png', dpi=300)
    print(f"Figure saved to: {figure_path}")
plt.show()